Here we want to develop code to perform bespoke deduplication using the output from ``umi_tools group``

In [42]:
import pysam
import collections
import copy
from time import gmtime, strftime

In [11]:
infile = "/home/FILESERVER5/proteomics/tss38/git_repos/cell_barcode/run/mapped/hgmm_100_hg_mm_grouped.bam"

In [23]:
read_ids = set()
gene2reads = collections.defaultdict(set)

for read in pysam.Samfile(infile, "rb").fetch():
    read_name = read.query_name
    read_id, cell, umi = read_name.split("_")
    
    # remove the redundant first three sections to save space
    short_read_id = ":".join(read_id.split(":")[3:]) 
    
    try:
        gene = read.get_tag("XT")
    except:
        continue # no gene assigned

    read_ids.add(short_read_id)
    gene2reads[gene].add(short_read_id)

In [21]:
print(read)

ST-K00126:308:HFLYFBBXX:6:1211:6532:28991_TGGCCAGCACCTTGTC_ACTAGCGAGC	272	0	182539	1	98M	-1	-1	98	CCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTGAGCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGGGATGGGCCATTGTTC	array('B', [41, 41, 41, 41, 41, 41, 41, 41, 37, 41, 37, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 37, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 37, 41, 41, 41, 41, 41, 41, 41, 37, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 37, 41, 41, 41, 41, 32, 37, 32, 41, 41, 37, 41, 41, 37, 41, 41, 37, 37, 32, 27, 32])	[('AS', 0), ('ZS', 0), ('XN', 0), ('XM', 0), ('XO', 0), ('XG', 0), ('NM', 0), ('MD', '98'), ('YT', 'UU'), ('NH', 8), ('XS', 'Assigned'), ('XN', 1), ('XT', 'ENSG00000279928'), ('UG', 171959), ('BX', 'ACTAGCGAGC')]


In [24]:
print(len(read_ids))
print(len(gene2reads))
print(len(gene2readCounts))

4204636
24323
24323


In [53]:
keep_genes = set()
read_ids2 = copy.copy(read_ids)
gene2reads2 =  copy.copy(gene2reads)
n = 0
while len(read_ids2) > 0:
    #if n >= 100:
    #    break
    n+=1
    
    top_gene = None
    top_gene_count = 0
    
    for gene in gene2reads2.keys():
        if len(gene2reads2[gene]) > top_gene_count:
            top_gene_count = len(gene2reads2[gene])
            top_gene = gene
    
    keep_genes.add(top_gene)
    removed_reads = gene2reads2[top_gene]
    
    read_ids2 = read_ids2.difference(removed_reads)
    
    if n%1000 == 0 or (n < 1000 and n%100 == 0):
        print(n, top_gene, len(gene2reads2[top_gene]), len(removed_reads),
              len(read_ids2), len(gene2reads2), strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    
    del gene2reads2[top_gene]
    
    for gene in gene2reads2.keys():
        #if gene2reads2[gene].difference(removed_reads) != gene2reads2[gene]:
        #    print(n)
        #    print(gene, top_gene, len(gene2reads2[gene]), len(gene2reads2[gene].difference(removed_reads)))
        #    raise ValueError()
        gene2reads2[gene] = gene2reads2[gene].difference(removed_reads)

    #if len(read_ids2)<1000:
    #    break
        
print(n)

100 ENSMUSG00000061477 7721 7721 2893059 24225 2017-08-23 08:44:53
200 ENSMUSG00000025967 3576 3576 2365665 24125 2017-08-23 08:45:27
300 ENSG00000104529 2065 2065 2094963 24025 2017-08-23 08:45:56
400 ENSG00000113387 1476 1476 1923739 23925 2017-08-23 08:46:19
500 ENSG00000085662 1172 1172 1793232 23825 2017-08-23 08:46:43
600 ENSG00000130204 957 957 1687772 23725 2017-08-23 08:47:04
700 ENSMUSG00000063694 828 828 1598531 23625 2017-08-23 08:47:25
800 ENSG00000178127 725 725 1520963 23525 2017-08-23 08:47:45
900 ENSMUSG00000064326 654 654 1452139 23425 2017-08-23 08:48:04
1000 ENSG00000241685 593 593 1389826 23325 2017-08-23 08:48:23
2000 ENSG00000031698 292 292 984199 22325 2017-08-23 08:51:06
3000 ENSG00000147419 188 188 751032 21325 2017-08-23 08:53:03
4000 ENSG00000174684 135 135 591262 20325 2017-08-23 08:54:40
5000 ENSG00000072958 102 102 474055 19325 2017-08-23 08:56:02
6000 ENSG00000163848 79 79 383971 18325 2017-08-23 08:57:11
7000 ENSG00000153094 64 64 312711 17325 2017-08-2